<a href="https://colab.research.google.com/github/SELF-msselve/UTN/blob/main/CeL_Procesamiento_tiempo_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install confluent_kafka
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.7 MB/s eta 0:00:00


In [ ]:
import json
from datetime import timezone
from random import choice, randint
from time import sleep

from confluent_kafka import Producer
from faker import Faker

In [ ]:
fake = Faker()

kafka_config = {
    'bootstrap.servers': '',
    'sasl.mechanism': 'SCRAM-SHA-256',
    'security.protocol': 'SASL_SSL',
    'sasl.username': '',
    'sasl.password': ''
}
producer = Producer(kafka_config)

KafkaException: KafkaError{code=_INVALID_ARG,val=-186,str="Failed to create producer: sasl.username and sasl.password must be set"}

In [ ]:
def generate_fake_log(fake):
    """
    Generar un registro de log ficticio

    Args:
        fake: instancia de Faker
    Returns:
        str: registro de log ficticio
    """
    ip = fake.ipv4()
    tstamp = fake.date_time_between(
        start_date='-1m', end_date='now', tzinfo=timezone.utc)
    request = f"{choice(['GET', 'POST'])} {fake.uri_path()}"
    status_code = choice([200, 401, 403, 404, 500])
    size = randint(2**4, 2**16)
    log = f'{tstamp.strftime("%Y-%m-%d %H:%M:%S")} - {ip} - {request} - {status_code} - {size}'
    return log

def generate_fake_transaction(fake):
    """
    Generar una transacción ficticia de tarjeta de crédito

    Args:
        fake: instancia de Faker
    Returns:
        dict: transacción ficticia
    """
    transaction = {
        'cc_no': fake.credit_card_number(card_type=None),
        'cc_provider': fake.credit_card_provider(card_type=None),
        'cc_expire': fake.credit_card_expire(start='now', end='+10y', date_format='%m/%y'),
        'amount': randint(10, 1000),
        'currency': 'USD',
        'merchant': fake.company(),
        'timestamp': fake.date_time_between(start_date='-6m', end_date='now', tzinfo=timezone.utc).isoformat()
    }
    return transaction

def delivery_callback(err, msg):
    if err is not None:
        print(f'Message failed delivery: {err}')
    else:
        print(f'Message delivered to {msg.topic()}: {msg.value()}')

In [ ]:
generate_fake_log(fake)

'2024-03-27 01:30:27 - 170.159.108.35 - GET tag - 200 - 11562'

In [ ]:
generate_fake_transaction(fake)

{'cc_no': '180062771891530',
 'cc_provider': 'VISA 16 digit',
 'cc_expire': '03/29',
 'amount': 894,
 'currency': 'USD',
 'merchant': 'Meadows-Edwards',
 'timestamp': '2024-03-27T01:26:30.270890+00:00'}

In [ ]:
for i in range(randint(5, 10)):
    log_record = generate_fake_log(fake)
    producer.produce(
        'web_server_logs',
        log_record.encode('utf-8'),
        callback=delivery_callback
    )
    producer.poll(0)

Message delivered to web_server_logs: b'2024-03-24 22:49:15 - 109.41.168.219 - GET posts - 404 - 40221'
Message delivered to web_server_logs: b'2024-03-24 22:49:33 - 32.236.62.179 - GET main/app/wp-content - 401 - 34458'
Message delivered to web_server_logs: b'2024-03-24 22:49:28 - 105.137.29.239 - PUT posts/category/app - 403 - 18663'
Message delivered to web_server_logs: b'2024-03-24 22:49:20 - 103.227.38.178 - GET categories/list/list - 200 - 46758'
Message delivered to web_server_logs: b'2024-03-24 22:48:55 - 34.69.234.109 - POST categories/wp-content/search - 200 - 50072'
Message delivered to web_server_logs: b'2024-03-24 22:48:59 - 86.142.213.12 - POST app/list/main - 403 - 509'
Message delivered to web_server_logs: b'2024-03-24 22:49:03 - 213.229.25.22 - DETELE list - 403 - 56478'
Message delivered to web_server_logs: b'2024-03-24 22:48:53 - 222.192.73.179 - POST category/blog - 401 - 46476'


In [ ]:
for i in range(randint(1, 5)):
    transaction_record = generate_fake_transaction(fake)
    producer.produce(
        'credit_card_transactions',
        json.dumps(transaction_record).encode('utf-8'),
        callback=delivery_callback
    )
    producer.poll(0)